# Genetic algorithms tested on Frozen Lake 

This tutorial will use the [OpenAI gym](https://gym.openai.com/) to test an evolutionary computation algorithm. Gym is a toolkit for developing and comparing reinforcement learning and genetic algorithms. It makes no assumptions about the structure of your agent, and is compatible with, for example, TensorFlow. We will later use it to study reinforcement learning. 

The gym is a collection of test problems -- environments -- that you can use to work out your algorithms. These environments have a shared interface, allowing you to write general algorithms.

## Installation
If you run this on your local machine then you will probably need to install ```gym```, however on ```gym``` seems to be pre-installed on Colab.
```
pip install gym
```

## The Frozen Lake problem

Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend.

The surface is described using a grid like the following:

    SFFF       (S: starting point, safe)
    FHFH       (F: frozen surface, safe)
    FFFH       (H: hole, fall to your doom)
    HFFG       (G: goal, where the frisbee is located)
    
The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.

Both the set of actions and states are discrete sets. The set of states ```S``` includes the possible locations in the grid (1 to 16). The set of actions ```A``` includes possibles moves: 

    LEFT = 0
    DOWN = 1
    RIGHT = 2
    UP = 3


See the Frozen Lake problem at [the OpenAI gym](https://gym.openai.com/envs/FrozenLake-v0/).

### Question 1. 
In every position/state you have 4 possible actions. Also, assume that you pass through every state on your way to the goal. That is, from start (```S```) to goal (```G```) is a sequence of 16 states (including the end points).

A set of actions given the states is called a **policy**.

* How many possible policies are there in Frozen Lake?

Assume that it takes approximately 1 ms to evaluate each sequence.

* How many hours would it take to find the best policy via a brute force search?
* How is the Frozen Lake problem related to planning?

## Take a look at the gym environment


In [3]:
import gym
# get an instance of the FrozenLake environment
env = gym.make('FrozenLake-v1')
# reset to start state
env.reset()
# render the env for inspection
env.render()
# the square with red? background color is where you're currently located.


SFFF
FHFH
FFFH
HFFG


In [14]:
env.reset()
env.step(3)
env.render()

  (Up)
SFFF
FHFH
FFFH
HFFG


* Take an action ```env.step(action)```, render again and observe the behavior.

### Question 2. 
* ```env.step()``` returns some values. What are those?

### Question 3. 
* Take and render multiple steps. Is the environment deterministic or stochastic?

## Random search

The cell below generates a set of 2000 random solutions and evaluates them. Depending on your answer above, you might want to do a brute force search; just change the variable ```n_policies``` below.

### Question 4. 
Run the cell 10 times and record the best score each time.
What is the best score you can got?

In [17]:
import numpy as np
import time
import gym

def run_episode(env, policy, episode_len=100):
    """
    env         : an instance of the gym environment "FrozenLake-v0"
    policy      : a set of actions to perform in a given state/position
    episode_len : upper limit on how many steps/actions can be taken
    """
    total_reward = 0.
    obs = env.reset()
    for t in range(episode_len):
        action = policy[obs]
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, n_episodes=100):
    """
    Runs n_episodes of a policy and returns the average reward.
    
    env         : an instance of the gym environment "FrozenLake-v0"
    policy      : a set of actions to perform in a given state/position
    n_episodes  : the number of episodes to run the policy
    """
    total_rewards = 0.0
    for _ in range(n_episodes):
        total_rewards += run_episode(env, policy)
    return total_rewards / n_episodes

def gen_random_policy():
    """
    Generates a random policy of 4 actions in 16 states/position
    """
    return np.random.choice(4, size=((16)))


env = gym.make('FrozenLake-v1')
## Policy search
n_policies = 2000
start = time.time()
policy_set = []
for _ in range(n_policies):
    policy_set.append(gen_random_policy())
policy_scores = [evaluate_policy(env, p) for p in policy_set]
end = time.time()

print("Best score = %0.2f. Time taken = %4.4f seconds" % (np.max(policy_scores) , end - start))

Best score = 0.29. Time taken = 48.0800 seconds
Best score = 0.39. Time taken = 40.4485 seconds


## Genetic Algorithm -- ONLY Crossover

Improve your random search with crossover.

### Question 5.
Implement the function ```crossover()``` in the cell below. Tip: try to adapt code from the lecture notes or slides.

* What score do you get now?

In [19]:
import numpy as np
import random
import time
import gym
from gym import wrappers

def run_episode(env, policy, episode_len=100):
    """
    Same as above
    """
    total_reward = 0
    obs = env.reset()
    for t in range(episode_len):
        action = policy[obs]
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, n_episodes=100):
    """
    Same as above
    """    
    total_rewards = 0.0
    for _ in range(n_episodes):
        total_rewards += run_episode(env, policy)
    return total_rewards / n_episodes

def gen_random_policy():
    """
    Same as above
    """    
    return np.random.choice(4, size=((16)))

def crossover(policy1, policy2):
    """
    Single Point Random Crossover

    Arguments
    ----------
    policy1: parent 1
    policy2: parent 2

    Return
    --------
    child_policy: offspring
    """
    child_policy = policy1.copy()
    j = np.random.randint(1, len(policy1) -2)
    child_policy[j:] = policy2[j:]
    
    # IMPLEMENT!
    # generate a child policy from cross-over of the parents

    return child_policy

random.seed(1)
np.random.seed(1)
env = gym.make('FrozenLake-v1')
env.seed(0)

## Policy search
n_policy = 100
n_gens = 20 # number of generations
start = time.time()

# initialize the population with random policies, i.e. a policy population
policy_pop = []
for _ in range(n_policy):
    policy_pop.append(gen_random_policy())

for gen in range(1, n_gens+1):
    
    # evaluate fitness of the policies
    policy_scores = []
    for policy in policy_pop:
        policy_scores.append(evaluate_policy(env, policy))    

    # Print the best score of each generation to see improvement
    print('Generation %d: max score = %0.2f' %(gen, max(policy_scores)))
    
    # rank the policies based on fitness
    policy_ranks = list(reversed(np.argsort(policy_scores)))
    # select the top 5 policies: no cross-over
    elite_set = [policy_pop[x] for x in policy_ranks[:5]]
    # get selection probabilities based on the policy scores
    select_probs = np.array(policy_scores) / np.sum(policy_scores)
    
    child_set = []
    for i in range(n_policy - 5):
        # randomly select parents weighted towards high select_probs
        parent1 = policy_pop[np.random.choice(range(n_policy), p=select_probs)]
        parent2 = policy_pop[np.random.choice(range(n_policy), p=select_probs)]
        # generate a child from cross-over btw parents
        child = crossover(parent1, parent2)
        child_set.append(child)  

    # add the elite set (no cross-over) to the new policy population
    policy_pop = elite_set
    # add the child set
    policy_pop += child_set

# Get the final policy scores after n_gens of selection and cross-over
policy_scores = []
for policy in policy_pop:
    policy_scores.append(evaluate_policy(env, policy))
    
# Select the best policy
best_policy = policy_pop[np.argmax(policy_scores)]

end = time.time()
print('Best policy score = %0.2f. Time taken = %4.4f' %(np.max(policy_scores), (end-start)))    

## Evaluation
env = wrappers.Monitor(env, '/tmp/frozenlake1', force=True)
for _ in range(200):
    run_episode(env, best_policy)
env.close()

Generation 1: max score = 0.26
Generation 2: max score = 0.33
Generation 3: max score = 0.41
Generation 4: max score = 0.47
Generation 5: max score = 0.52
Generation 6: max score = 0.55
Generation 7: max score = 0.66
Generation 8: max score = 0.71
Generation 9: max score = 0.66
Generation 10: max score = 0.66
Generation 11: max score = 0.72
Generation 12: max score = 0.67
Generation 13: max score = 0.73
Generation 14: max score = 0.71
Generation 15: max score = 0.71
Generation 16: max score = 0.72
Generation 17: max score = 0.72
Generation 18: max score = 0.71
Generation 19: max score = 0.72
Generation 20: max score = 0.72
Best policy score = 0.72. Time taken = 116.8944


## Genetic Algorithm -- Crossover AND Mutation

### Question 5.
* Implement the function ```mutation()``` in the code below. You can implement either flip, swap, scramble or inversion.

* Does the performance increase further?

In [3]:
import numpy as np
import random
import time
import gym
from gym import wrappers

def run_episode(env, policy, episode_len=100):
    total_reward = 0
    obs = env.reset()
    for t in range(episode_len):
        action = policy[obs]
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
            
    return total_reward


def evaluate_policy(env, policy, n_episodes=100):
    
    total_rewards = 0.0
    for _ in range(n_episodes):
        total_rewards += run_episode(env, policy)
        
    return total_rewards / n_episodes


def gen_random_policy():
    
    return np.random.choice(4, size=((16)))


def crossover(policy1, policy2):
    """
    single point random crossover
    Arguments
    ----------
    policy1: parent 1
    policy2: parent 2

    Return
    --------
    child_policy: offspring
    """
    child_policy = policy1.copy()
    # IMPLEMENT!
    # generate a child policy from cross-over of the parents
    j = np.random.randint(1, len(policy1) -2)
    child_policy[j:] = policy2[j:]

    return child_policy


def mutation(policy, symbols=[0, 1, 2, 3], p=0.05):
    """
    swap mutation
    Arguments
    ---------
    policy -- policy to be mutated
    p      -- mutation probability

    Return
    ------
    mutated_policy -- a mutated version of 'policy'
    """
    #mutated_policy = policy.copy()
    ids = np.random.randint(1, len(policy) -1, 2)
    policy[ids[0]], policy[ids[1]] = policy[ids[1]], policy[ids[0]]
    # IMPLEMENT!
    # generate a mutated policy.    
            
    return policy

#set gym env
random.seed(1)
np.random.seed(1)
env = gym.make('FrozenLake-v1')
env.seed(0)

## Policy search
n_policy = 100
n_gens = 20 # number of generations
start = time.time()

# initialize the population with random policies, i.e. a policy population
policy_pop = []
for _ in range(n_policy):
    policy_pop.append(gen_random_policy())

for gen in range(1, n_gens+1):
    
    # evaluate fitness of the policies
    policy_scores = []
    for policy in policy_pop:
        policy_scores.append(evaluate_policy(env, policy))    
        
    # Print the best score of each generation to see improvement
    print('Generation %d: max score = %0.2f' %(gen, max(policy_scores)))
    
    # rank the policies based on fitness
    policy_ranks = list(reversed(np.argsort(policy_scores)))
    # select the top 5 policies: no cross-over
    elite_set = [policy_pop[x] for x in policy_ranks[:5]]
    # get selection probabilities based on the policy scores
    select_probs = np.array(policy_scores) / np.sum(policy_scores)
    
    
    child_set = []
    for i in range(n_policy - 5):
        # randomly select parents weighted towards high select_probs
        parent1 = policy_pop[np.random.choice(range(n_policy), p=select_probs)]
        parent2 = policy_pop[np.random.choice(range(n_policy), p=select_probs)]
        # generate a child from cross-over btw parents
        child = crossover(parent1, parent2)
        child_set.append(child)  
        
    # mutate the children
    mutated_list = []
    for child in child_set:
        mutated = mutation(child)
        mutated_list.append(mutated)

    # add the elite set (no cross-over) to the new policy population
    policy_pop = elite_set
        # add the mutated child set
    policy_pop += mutated_list

policy_scores = []
for policy in policy_pop:
    policy_scores.append(evaluate_policy(env, policy))
    
best_policy = policy_pop[np.argmax(policy_scores)]

end = time.time()
print('Best policy score = %0.2f. Time taken = %4.4f'
        %(np.max(policy_scores), (end-start)))    

## Evaluation
env = wrappers.Monitor(env, '/tmp/frozenlake1', force=True)
for _ in range(200):
    run_episode(env, best_policy)
env.close()

Generation 1: max score = 0.26
Generation 2: max score = 0.26
Generation 3: max score = 0.48
Generation 4: max score = 0.58
Generation 5: max score = 0.65
Generation 6: max score = 0.62
Generation 7: max score = 0.80
Generation 8: max score = 0.73
Generation 9: max score = 0.78
Generation 10: max score = 0.80
Generation 11: max score = 0.78
Generation 12: max score = 0.79
Generation 13: max score = 0.78
Generation 14: max score = 0.76
Generation 15: max score = 0.77
Generation 16: max score = 0.77
Generation 17: max score = 0.81
Generation 18: max score = 0.80
Generation 19: max score = 0.78
Generation 20: max score = 0.80
Best policy score = 0.80. Time taken = 112.5857


## Genetic Algorithm -- Crossover AND Mutation AND diversity

### Question 6 (optional).
* Implement the function an alternative to ```evaluate_policy()``` called ```evaluate_policy_diversity_fitness()``` that combines genetic diversity (see lecture slides) with fitness. Keep ```evaluate_policy()``` but rename it to ```evaluate_policy_fitness()```, it'll be useful for the final evaluation.
* Does the performance increase further?

In [16]:
import numpy as np
import random
import time
import gym
from gym import wrappers

def run_episode(env, policy, episode_len=100):
    total_reward = 0
    obs = env.reset()
    for t in range(episode_len):
        action = policy[obs]
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward


def mean_squared_error(policy1, policy2):
    """
    calculate the mean squared error between two policies
    :param policy1:
    :param policy2:
    :return:
    """
    delta = np.subtract(policy1, policy2)
    return np.square(delta).mean()


def max_mean_squared_error(prev_policy_pop):
    """
    get the maximum mean squared error for a policy group.
    to be used in normalize diversity score
    :param prev_policy_pop:
    :return:
    """
    total = 0.0
    for p in prev_policy_pop:
        total += np.square(p).mean()
    return total

def evaluate_diversity(policy, prev_policy_pop):
    """
    calculate the diversity score of a policy against a policy population using simple mean squared error
    :param policy:  targeted policy
    :param prev_policy_pop:  policy population
    :return: normalized diversity score
    """
    total = 0.0
    for p in prev_policy_pop:
        total += mean_squared_error(policy, p)
    return total / max_mean_squared_error(prev_policy_pop)


def evaluate_policy_diversity_fitness(env, policy, prev_policy_pop):
    """
    evaluate both the fitness and diversity of a policy with coefficient = 0.5
    score  = 0.5 fitness + 0.5 diversity
    """
    fitness = evaluate_policy_fitness(env, policy)
    diversity = evaluate_diversity(policy, prev_policy_pop)
    return 0.5 * fitness + 0.5 * diversity


def evaluate_policy_fitness(env, policy, n_episodes=100):
    
    total_rewards = 0.0
    for _ in range(n_episodes):
        total_rewards += run_episode(env, policy)
        
    return total_rewards / n_episodes


def gen_random_policy():
    
    return np.random.choice(4, size=((16)))


def crossover(policy1, policy2):
    """
    Arguments
    ----------
    policy1: parent 1
    policy2: parent 2

    Return
    --------
    child_policy: offspring
    """
    child_policy = policy1.copy()
    # IMPLEMENT!
    # generate a child policy from cross-over of the parents
    j = np.random.randint(1, len(policy1) -2)
    child_policy[j:] = policy2[j:]
    return child_policy


def mutation(policy, p=0.05):
    """
    Arguments
    ---------
    policy -- policy to be mutated
    p      -- mutation probability

    Return
    ------
    new_policy -- a mutated version of 'policy'
    """

    # IMPLEMENT!
    # mutate new_policy  
    ids = np.random.randint(1, len(policy) -1, 2)
    policy[ids[0]], policy[ids[1]] = policy[ids[1]], policy[ids[0]]

    return policy

#set gym env
random.seed(1)
np.random.seed(1)
env = gym.make('FrozenLake-v1')
env.seed(0)

## Policy search
n_policy = 100
n_gens = 20 # number of generations
start = time.time()

# initialize the population with random policies, i.e. a policy population
policy_pop = []
for _ in range(n_policy):
    policy_pop.append(gen_random_policy())

# to initialize previous policy population, just use the initial policy_pop
prev_policy_pop = policy_pop.copy()

for gen in range(1, n_gens+1):
    
    # evaluate fitness AND diversity of the policies
    policy_scores = []
    for policy in policy_pop:
        policy_scores.append(evaluate_policy_diversity_fitness(env, policy, prev_policy_pop))    
        
    # Print the best score of each generation to see improvement
    print('Generation %d: max fitness & diversity score = %0.2f' %(gen, max(policy_scores)))
    
    # rank the policies based on fitness
    policy_ranks = list(reversed(np.argsort(policy_scores)))
    # select the top 5 policies: no cross-over
    elite_set = [policy_pop[x] for x in policy_ranks[:5]]
    # get selection probabilities based on the policy scores
    select_probs = np.array(policy_scores) / np.sum(policy_scores)
    
    
    child_set = []
    for i in range(n_policy - 5):
        # randomly select parents weighted towards high select_probs
        parent1 = policy_pop[np.random.choice(range(n_policy), p=select_probs)]
        parent2 = policy_pop[np.random.choice(range(n_policy), p=select_probs)]
        # generate a child from cross-over btw parents
        child = crossover(parent1, parent2)
        child_set.append(child)  
        
    # mutate the children
    mutated_list = []
    for child in child_set:
        mutated = mutation(child)
        mutated_list.append(mutated)

    # add the elite set (no cross-over) to the new policy population
    policy_pop = elite_set
    # add the mutated child set
    policy_pop += mutated_list

# evaluate policies only on fitness
policy_scores = []
for policy in policy_pop:
    policy_scores.append(evaluate_policy_fitness(env, policy))
    
best_policy = policy_pop[np.argmax(policy_scores)]

end = time.time()
print('Best policy fitness score = %0.2f. Time taken = %4.4f'
        %(np.max(policy_scores), (end-start)))    

## Evaluation
env = wrappers.Monitor(env, '/tmp/frozenlake1', force=True)
for _ in range(200):
    run_episode(env, best_policy)
env.close()

Generation 1: max fitness & diversity score = 0.49
Generation 2: max fitness & diversity score = 0.49
Generation 3: max fitness & diversity score = 0.48
Generation 4: max fitness & diversity score = 0.46
Generation 5: max fitness & diversity score = 0.47
Generation 6: max fitness & diversity score = 0.48
Generation 7: max fitness & diversity score = 0.48
Generation 8: max fitness & diversity score = 0.70
Generation 9: max fitness & diversity score = 0.69
Generation 10: max fitness & diversity score = 0.71
Generation 11: max fitness & diversity score = 0.69
Generation 12: max fitness & diversity score = 0.72
Generation 13: max fitness & diversity score = 0.77
Generation 14: max fitness & diversity score = 0.78
Generation 15: max fitness & diversity score = 0.79
Generation 16: max fitness & diversity score = 0.81
Generation 17: max fitness & diversity score = 0.79
Generation 18: max fitness & diversity score = 0.80
Generation 19: max fitness & diversity score = 0.80
Generation 20: max fi